# Stacking 实现
## 一、使用的数据集
处理后的原训练集 特征数据：dg_pca_train<br />
处理后的原训练集合 得分数据：score_train<br />
处理后的测试集合：dg_pca_test<br />
测试集序号：id_test<br />


In [1]:
#预设导入
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import joblib

import warnings
warnings.filterwarnings("ignore")

In [17]:
#机器学习导入
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn import svm
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import explained_variance_score 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import r2_score
from sklearn import ensemble
from sklearn import neighbors

In [3]:
#读取本地数据集
dg_pca_train=pd.read_csv('dg_pca_train.csv').values
dg_pca_test=pd.read_csv('dg_pca_test.csv').values

id_test=pd.read_csv('id_test.csv').values
id_train=pd.read_csv('id_train.csv').values
score_train=pd.read_csv('score_train.csv').values
id_train=id_train.flatten()
id_test=id_test.flatten()#ID二维数组转一维

## stacking过程

### 新数据集
X:处理后的原训练集 特征数据：dg_pca_train<br />
y:处理后的原训练集 得分数据：score_train<br />
X_predict:处理后的测试集合 特征数据：dg_pca_test<br />
y_predict:NA<br />




In [5]:
# '''创建训练的数据集'''
#data, target = make_blobs(n_samples=50000, centers=2, random_state=0, cluster_std=0.60)
 
# '''模型融合中使用到的各个单模型'''
clfs = [RandomForestRegressor(n_estimators=500),
        svm.SVR(kernel='rbf',C=10,gamma=0.0001),
        XGBRegressor(max_depth=3,min_child_weight=5,learning_rate=0.1,n_estimators=1200,random_state=161)]
 
#'''切分一部分数据作为测试集'''
X=dg_pca_train
X_predict=dg_pca_test
y=score_train
#y_predict = 


dataset_blend_train = np.zeros((X.shape[0], len(clfs)))#第一轮 保存各个模型在训练集上的预测结果 训练集合个数×模型数
dataset_blend_test = np.zeros((X_predict.shape[0], len(clfs)))#第一轮 保存各个模型在测试集上的预测结果 训练集合个数×模型数

#'''5折stacking'''
n_folds = 5
kf = KFold(n_folds,True,50)
skf=list(kf.split(X))#X或者y

for j, clf in enumerate(clfs):
    #'''依次训练各个单模型'''
    print(j, clf)
    dataset_blend_test_j = np.zeros((X_predict.shape[0], len(skf)))#存目前这个模型上的测试集结果(之后求平均)
    for i, (train, test) in enumerate(skf):
        #'''使用第i个部分作为预测，剩余的部分来训练模型，获得其预测的输出作为第i部分的新特征。'''
        print("Fold", i)
        X_train, y_train, X_test, y_test = X[train], y[train], X[test], y[test]
        clf.fit(X_train, y_train)
        y_submission = clf.predict(X_test)#1fold的预测结果
        
        #y_submission.reshape(len(y_submission),1)#一维数组转二维 可以不加
        
        dataset_blend_train[test, j] = y_submission#在模型顺序对应的j位置 存1fold的预测结果
        dataset_blend_test_j[:, i] = clf.predict(X_predict)#存该模型该折下的测试集预测结果
        
    #'''对于测试集，直接用这k个模型的预测值均值作为新的特征。'''
    dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)#测试集结果按行取平均后储存
    
    
    #print("val auc Score: %f" % r2_score(y_predict, dataset_blend_test[:, j]))
    
    #保存模型
    fl_name="stacking_{0}".format(j)
    joblib.dump(clf, fl_name)


0 RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
1 SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.0001,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
2 XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=5, missing=None, n_estimators=1200,
             n_jobs=1, nthread

In [13]:
dataset_blend_train_1=dataset_blend_train
dataset_blend_test_1=dataset_blend_test

In [29]:
# '''创建训练的数据集'''
#data, target = make_blobs(n_samples=50000, centers=2, random_state=0, cluster_std=0.60)
 
# '''模型融合中使用到的各个单模型'''
clfs = [neighbors.KNeighborsRegressor(n_neighbors=9, p=2,weights='distance'),
        ensemble.GradientBoostingRegressor(n_estimators=500)]
 
#'''切分一部分数据作为测试集'''
X=dg_pca_train
X_predict=dg_pca_test
y=score_train
#y_predict = 


dataset_blend_train_2 = np.zeros((X.shape[0], len(clfs)))#第一轮 保存各个模型在训练集上的预测结果 训练集合个数×模型数
dataset_blend_test_2 = np.zeros((X_predict.shape[0], len(clfs)))#第一轮 保存各个模型在测试集上的预测结果 训练集合个数×模型数

#'''5折stacking'''
n_folds = 5
kf = KFold(n_folds,True,50)
skf=list(kf.split(X))#X或者y

for j, clf in enumerate(clfs):
    #'''依次训练各个单模型'''
    print(j, clf)
    dataset_blend_test_j = np.zeros((X_predict.shape[0], len(skf)))#存目前这个模型上的测试集结果(之后求平均)
    for i, (train, test) in enumerate(skf):
        #'''使用第i个部分作为预测，剩余的部分来训练模型，获得其预测的输出作为第i部分的新特征。'''
        print("Fold", i)
        X_train, y_train, X_test, y_test = X[train], y[train], X[test], y[test]
        clf.fit(X_train, y_train)
        y_submission = clf.predict(X_test)#1fold的预测结果
        y_submission=y_submission.flatten()#2维数组转1维 KNN需要 
        #y_submission.reshape(len(y_submission),1)#一维数组转二维 可以不加
        
        dataset_blend_train_2[test, j] = y_submission#在模型顺序对应的j位置 存1fold的预测结果
        dataset_blend_test_j[:, i] = clf.predict(X_predict).flatten()#存该模型该折下的测试集预测结果
        
    #'''对于测试集，直接用这k个模型的预测值均值作为新的特征。'''
    dataset_blend_test_2[:, j] = dataset_blend_test_j.mean(1)#测试集结果按行取平均后储存
    
    
    #print("val auc Score: %f" % r2_score(y_predict, dataset_blend_test[:, j]))
    
    #保存模型
    fl_name="stacking_{0}".format(j+2)
    joblib.dump(clf, fl_name)

0 KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                    weights='distance')
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
1 GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=500,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4


In [22]:
dataset_blend_train_2=dataset_blend_train_2[:,0]

In [30]:
#导出数据集到本地
submission_train_1=pd.DataFrame(dataset_blend_train_2)
#submission_train_1.head()
submission_train_1.to_csv('dataset_blend_train_2.csv',index=False)#第一轮训练后 train集合预测得到的score集合 训练集样本数x3个模型

submission_test_1=pd.DataFrame(dataset_blend_test_2)
#submission_test_1.head()
submission_test_1.to_csv('dataset_blend_test_2.csv',index=False)#第一轮训练后 test集合预测得到的score集合 测试机样本数x3个模型

In [40]:
#拼合数据
dataset_blend_test=np.hstack((dataset_blend_test_1,dataset_blend_test_2))
dataset_blend_train=np.hstack((dataset_blend_train_1,dataset_blend_train_2))

In [41]:
dataset_blend_test

array([[1.55434901, 1.71705025, 1.72925215, 1.30136768, 1.70438694],
       [2.32998628, 2.6439508 , 2.60908222, 2.41936645, 2.59386453],
       [2.49372005, 2.33086505, 2.69724541, 2.64994564, 2.67796928],
       ...,
       [1.88082266, 2.31558535, 1.82576983, 2.25811547, 2.07382897],
       [2.36509145, 2.30614058, 2.31984725, 2.2470369 , 2.26451797],
       [2.50106031, 2.5574335 , 2.54165936, 2.56390571, 2.57053153]])

In [43]:
dataset_blend_test_2

array([[1.30136768, 1.70438694],
       [2.41936645, 2.59386453],
       [2.64994564, 2.67796928],
       ...,
       [2.25811547, 2.07382897],
       [2.2470369 , 2.26451797],
       [2.56390571, 2.57053153]])

In [46]:
#第二轮
clf = svm.SVR(C=1.0,gamma=0.1)

clf.fit(dataset_blend_train, y)
y_submission = clf.predict(dataset_blend_test)

#生成文件
submission_df=pd.DataFrame(data={'Id':id_test,'SalePrice':y_submission})
submission_df.head()
submission_df.to_csv('baseline_st5.csv',header=False,index=False)

In [37]:
dataset_blend_test_2

array([[1.30136768, 1.70438694],
       [2.41936645, 2.59386453],
       [2.64994564, 2.67796928],
       ...,
       [2.25811547, 2.07382897],
       [2.2470369 , 2.26451797],
       [2.56390571, 2.57053153]])

In [10]:
clf = svm.SVR()
print(clf)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)


In [45]:
#调参数
X_train, X_test, y_train, y_test = train_test_split(dataset_blend_train, y, test_size=0.2, random_state=123)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

#SVR模型调参数：C gamma
param_g=[{"C": [1e0, 1e1, 1e2, 1e3],"gamma": np.logspace(-4, 0, 5)}]
scores_g=['r2']
for score in scores_g:
    print(score)
    clf=GridSearchCV(svm.SVR(),param_grid=param_g,cv=3,scoring=score)
    clf.fit(X_train,y_train)
    
    print(clf.best_estimator_)
    
    means = clf.cv_results_['mean_test_score']
    stds=clf.cv_results_['std_test_score']
    params = clf.cv_results_['params']
    
    for (param,mean_score,std_score) in zip(params,means,stds):
        print("%0.3f(+/-%0.03f) for %r"%(mean_score,std_score,param))

(10984, 5) (10984, 1) (2747, 5) (2747, 1)
r2
SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
0.527(+/-0.008) for {'C': 1.0, 'gamma': 0.0001}
0.569(+/-0.007) for {'C': 1.0, 'gamma': 0.001}
0.574(+/-0.008) for {'C': 1.0, 'gamma': 0.01}
0.575(+/-0.009) for {'C': 1.0, 'gamma': 0.1}
0.569(+/-0.010) for {'C': 1.0, 'gamma': 1.0}
0.568(+/-0.007) for {'C': 10.0, 'gamma': 0.0001}
0.573(+/-0.008) for {'C': 10.0, 'gamma': 0.001}
0.574(+/-0.008) for {'C': 10.0, 'gamma': 0.01}
0.574(+/-0.009) for {'C': 10.0, 'gamma': 0.1}
0.557(+/-0.011) for {'C': 10.0, 'gamma': 1.0}
0.572(+/-0.008) for {'C': 100.0, 'gamma': 0.0001}
0.573(+/-0.008) for {'C': 100.0, 'gamma': 0.001}
0.575(+/-0.008) for {'C': 100.0, 'gamma': 0.01}
0.572(+/-0.008) for {'C': 100.0, 'gamma': 0.1}
0.520(+/-0.009) for {'C': 100.0, 'gamma': 1.0}
0.572(+/-0.008) for {'C': 1000.0, 'gamma': 0.0001}
0.573(+/-0.008) for {'C': 1000.0, 'gamma': 0.001}
0

In [9]:
# clfs = [RandomForestRegressor(),
#         svm.SVR(),
#         XGBRegressor()]
# for j, clf in enumerate(clfs):
#     fl_name="stacking_{0}".format(j)
#     joblib.dump(clf, fl_name)

In [ ]:
#在独立测试集上预测数据
svc_best_predict=svc_best.predict(dg_pca_test)
print(svc_best_predict)
np.shape(svc_best_predict)

In [50]:
print(clf.best_estimator_)


AttributeError: 'SVR' object has no attribute 'best_estimator_'